In [1]:
import warnings
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime


In [2]:
TEJ_FINANCE_STORE = "tej_finance.h5"
with pd.HDFStore(TEJ_FINANCE_STORE) as store:
    df = store['tej/price_adj_data'].loc[:,['年月日','公司','close']]
df=df.set_index(['年月日','公司'])
df.sort_index(level=['年月日','公司'], ascending=[1, 1], inplace=True)#必須
df=np.log(df.unstack('公司')/df.unstack('公司').shift(1)).stack('公司')
df=df.rename(columns={'close':'log_ret'})
df=df.dropna()
k1=pd.Grouper(level='公司')
df['roll_sd']=df.groupby(k1)['log_ret'].apply(lambda x: x.rolling(60).std())

const=pd.read_csv('0050_con.csv')
const['年月日']=pd.to_datetime(const['年月日'], format='%Y/%m/%d')
const['成份股']=[str(i).strip().replace(" ","_") for i in const['成份股']]
const=const[['年月日','成份股']]
const['tw_0050']=True
const=const.rename(columns={'成份股':'公司'})
const=const.set_index(['年月日','公司'])
const.sort_index(level=['年月日','公司'], ascending=[1, 1], inplace=True)
df=df.join(const).dropna()

factor=pd.read_csv('0050_con_tejfactor.csv')
factor['年月日']=pd.to_datetime(factor['年月日'], format='%Y/%m/%d')
factor=factor.set_index(['年月日'])
factor.sort_index(level=['年月日'],ascending=True, inplace=True)
df=df.join(factor)
df['log_ret']=df['log_ret']-0.01045
df=df.drop('tw_0050',axis=1)
df2=df.drop('log_ret',axis=1)
df=df[['log_ret']]
df2=df2.reset_index()
df=df.reset_index()
print(df.info())

#全年
df['count']=df.groupby('公司')['年月日'].transform('count')
df=df[df['count']==242]
df=df.reset_index().drop(['index','count'],axis=1)
df2['count']=df2.groupby('公司')['年月日'].transform('count')
df2=df2[df2['count']==242]
df2=df2.reset_index().drop(['index','count'],axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16974 entries, 0 to 16973
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   年月日      16974 non-null  datetime64[ns]
 1   公司       16974 non-null  object        
 2   log_ret  16974 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 398.0+ KB
None


In [3]:
from statsmodels.api import OLS, add_constant
X=[i for i in df2.columns if i not in['年月日','公司','count']]
comp=list(set(df['公司']))
betas = []
first_date=[]
for i in comp:
    step1 = OLS(endog=df[df['公司']==i]['log_ret'], 
                exog=add_constant(df2[df2['公司']==i][X])).fit()
    betas.append(step1.params.drop('const'))
    first_date.append(df[df['公司']==i]['年月日'].iloc[0])
betas = pd.DataFrame(betas, 
                    columns=X,index=set(df['公司']))
betas=betas.reset_index()
betas['first_date']=first_date
betas

,index,roll_sd,市場風險溢酬,規模溢酬 (5因子),淨值市價比溢酬,盈利能力因子,投資因子,first_date
0,3545_敦泰,-0.290326,2.350230,2.186475,0.506713,-0.635930,0.229596,2019-01-02
1,2369_菱生,0.235550,1.918993,1.721044,0.273810,-0.617841,0.682420,2019-01-02
2,3711_日月光投控,0.311800,1.286142,-0.564783,-0.514396,-0.313783,0.327709,2019-01-02
3,2436_偉詮電,0.341445,1.833872,1.618986,0.441981,-0.406480,0.359386,2019-01-02
4,5269_祥碩,0.045394,1.966740,1.936376,0.822835,0.547071,0.711008,2019-01-02
...,...,...,...,...,...,...,...,...
59,3006_晶豪科,1.007643,1.694706,1.324616,0.490923,-0.345596,-0.331471,2019-01-02
60,3014_聯陽,-0.053039,1.332761,1.271601,0.704384,0.471848,0.269612,2019-01-02
61,4952_凌通,0.144585,1.784657,1.510671,0.660265,-0.282668,0.271204,2019-01-02
62,6239_力成,0.347253,1.219728,-0.010345,0.527835,-0.123860,0.754885,2019-01-02


In [4]:
lambdas = []
lambdas2= []
date=list(set(df['年月日']))
date.sort()
count=0
for period in date:
    
    step2 = OLS(endog=list(df[df['年月日']==period]['log_ret']),exog=betas[X]).fit()
    lambdas.append(step2.params)
    lambdas2.append(step2.tvalues)
    print(count)
    count+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241


In [5]:
gamma=pd.DataFrame(lambdas,index=date,
                       columns=X)
gammat=pd.DataFrame(lambdas2,index=date,
                       columns=X)
gamma

,roll_sd,市場風險溢酬,規模溢酬 (5因子),淨值市價比溢酬,盈利能力因子,投資因子
2019-01-02,-0.003962,-0.014749,0.004971,0.000557,-0.000429,0.000583
2019-01-03,-0.009028,-0.016880,0.007089,-0.005079,0.005191,0.007100
2019-01-04,-0.013711,-0.020323,0.010083,-0.000384,-0.005045,0.014898
2019-01-07,0.003144,0.002019,0.000034,0.004083,-0.007839,0.001519
2019-01-08,-0.007500,-0.007692,0.004285,0.001624,-0.003567,0.007707
...,...,...,...,...,...,...
2019-12-25,0.007188,-0.003924,0.000946,0.002934,-0.002686,-0.006489
2019-12-26,-0.009545,-0.002230,-0.002321,0.003399,0.001240,-0.001527
2019-12-27,0.004128,-0.010288,0.012001,0.003232,0.001468,-0.001231
2019-12-30,-0.006181,-0.009999,0.003640,-0.001070,-0.002541,0.000573


In [6]:
gamma.mean()/(gamma.std()/np.sqrt(len(gamma)))

roll_sd       -4.437446
市場風險溢酬       -12.325820
規模溢酬 (5因子)     5.661019
淨值市價比溢酬       -3.167899
盈利能力因子        -0.929969
投資因子           2.488024
dtype: float64